# Web Scraping - Centadata (Aberdeen/Ap Lei Chau) - ver2.0

In [1]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

In [2]:
#Define Variable
DistrictName ="aberdeen.apleichau"
DistrictNameinChinese ="香港仔/鴨脷洲"
DistrictNameinChinese_Search ="南區"

OutFile = "../../../input/centadata_"+DistrictName+"_202106.csv"
cutoff_Date = pd.to_datetime("30/04/2021",format="%d/%m/%Y")

# set the URL for web scraping
url="http://hk.centadata.com/ptest.aspx?type=22&code=105&ref=CD2_Detail_Nav" #Aberdeen/Ap Lei Chau
#url="http://hk.centadata.com/ptest.aspx?type=22&code=102&ref=CD2_Detail_Nav"

In [3]:
# initialize the lists for storing data
build_names = []
phases = []
floors = []
blocks = []
txn_dates = []
build_ages = []
saleable_areas = []
txn_prices = []

In [4]:
#define for Stop Nested Loops
class GetOutOfLoop( Exception ):
    pass

In [5]:
# The chrome browser is saved in /usr/local/bin
#driver = webdriver.Chrome("C:\Users\chan\Downloads\chromedriver\chromedriver.exe") #start a chrome browserb

In [6]:
# The chrome browser is saved in /usr/local/bin
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\chan\.wdm\drivers\chromedriver\win32\96.0.4664.45]


In [7]:
driver.get(url) # go to the URL
time.sleep(8) # time.sleep(T) means wait for T seconds for the chrome browser to load the URL page

In [8]:
#for testing the field and button 
#i=8
#for i in range(8,16): # loop the transaction rows from 5 to 16
#    print(i)
#    web_txn_price = "//*[@id='chart2']/div[1]/table/tbody/tr[" + str(i) + "]/td[10]" 
#    print (web_txn_price)
#    txn_price = driver.find_element_by_xpath(web_txn_price) 
#    print(txn_price.text)
        
#屋苑
#web_build_name = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[3]"
#build_name = driver.find_element_by_xpath(web_build_name) 
#print(build_name.text)

#build_ages = []
#樓齡
#web_build_age = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[9]" 
#build_age = driver.find_element_by_xpath(web_build_age) 
#print(build_age.text)

#if build_age.text.find("萬") == -1:
#    build_ages.append(build_age.text)
#else:
#     build_ages.append(','+build_age.text)

#print(build_age)        

#期數        
#web_phase_name = "//*[@id='chart2']/div[1]/table/tbody/tr[5]/td[4]"
#phase = driver.find_element_by_xpath(web_phase_name) 
#print(phase.text)
   
#link = driver.find_element_by_xpath("//*[@id='chart2']/div[2]/div[1]/a[last()]")
#link.click() # click on "Next Page" link

In [9]:
# skip from page 90 to 181 
#for j in range(90,190, 10): # range(X,Y) means loop the page from X to Y-1 
#        link = driver.find_element_by_xpath("//*[@id='chart2']/div[2]/div[1]/a[@idx='"+str(j)+"']")
#        link.click()
#        time.sleep(90)
#        
#        link = driver.find_element_by_xpath("//*[@id='chart2']/div[2]/div[1]/a[@class='aRecordNext']")
#        link.click() # click on "Next Page" link
#        time.sleep(90)       

In [10]:
#web_txn_date = "//*[@id='chart2']/div[1]/table/tbody/tr[5]/td[1]" 
#txn_date = driver.find_element_by_xpath(web_txn_date) 
#d1 = pd.to_datetime(txn_date.text)
#d2 = pd.to_datetime("07/06/2021")
#print(txn_date.text)
#print (d1 <= d2)

In [16]:
#for j in range(1,2): # range(X,Y) means loop the page from X to Y-1 

# 4-May-2021 : because of 欣榮花園, using the txn_price to replace the build_age, 
#              if we find the "萬" in the build_age, then need to put the blank and modify the rest of field
               
for j in range(1,2000): # range(X,Y) means loop the page from X to Y-1 
    try:        
        for i in range(5,16): # loop the transaction rows from 5 to 16
            try:              
                #print(i)
                #註冊日期
                web_txn_date = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[1]" 
                txn_date = driver.find_element_by_xpath(web_txn_date) 
                #print(txn_date.text)
                if txn_date.text!="":
                    d1 = pd.to_datetime(txn_date.text,format="%d/%m/%Y")
                    if d1 <= cutoff_Date :
                        raise GetOutOfLoop 
                txn_dates.append(txn_date.text)

                #屋苑     
                web_build_name = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[3]"
                build_name = driver.find_element_by_xpath(web_build_name) 
                #print(build_name.text)
                build_names.append(build_name.text)
        
                #期數
                web_phase_name = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[4]"
                phase = driver.find_element_by_xpath(web_phase_name) 
                #print(phase.text)
                phases.append(phase.text)
                
                #樓齡
                web_build_age = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[9]" 
                build_age = driver.find_element_by_xpath(web_build_age) 
                #print(build_age.text)
                if build_age.text.find("萬") == -1:
                    build_ages.append(build_age.text)
                else:
                    build_ages.append("--")

                #成交價($)
                web_txn_price = "//*[@id='chart2']/div[1]/table/tbody/tr[" + str(i) + "]/td[10]" 
                txn_price = driver.find_element_by_xpath(web_txn_price) 
                #print(txn_price.text)
                if build_age.text.find("萬") == -1:
                    txn_prices.append(txn_price.text)
                else:
                    txn_prices.append(build_age.text)
                          
        
                #面積(呎) - 
                web_saleable_area = "//*[@id='chart2']/div[1]/table/tbody/tr[" + str(i) + "]/td[11]" 
                saleable_area = driver.find_element_by_xpath(web_saleable_area) 
                #print(saleable_area.text)
                if build_age.text.find("萬") == -1:
                    saleable_areas.append(saleable_area.text)
                else:
                    saleable_areas.append(txn_price.text)
                             
                #座數
                #web_block = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[5]" 
                #block = driver.find_element_by_xpath(web_block)
                #print(block.text)
                #blocks.append(block.text)
        
                #樓層
                web_floor = "//*[@id='chart2']/div[1]/table/tbody/tr["+str(i)+"]/td[6]" 
                floor = driver.find_element_by_xpath(web_floor)
                #print(floor.text)
                floors.append(floor.text)
        
            except NoSuchElementException:  #spelling error making this code not work as expected
                #print("End of Page")   
                break        

#    link = driver.find_element_by_xpath("//*[@id='chart2']/div[2]/div[1]/a[last()]")
        link = driver.find_element_by_xpath("//*[@id='chart2']/div[2]/div[1]/a[@class='aRecordNext']")
        link.click() # click on "Next Page" link
        time.sleep(40) # time.sleep(T) means wait for T seconds for the chrome browser to load the URL page
    
    except NoSuchElementException:  #spelling error making this code not work as expected
        #print("End of Section ")   
        break        

    except GetOutOfLoop:  # Break Out of Nested Loops
        break        

#print(txn_dates)
#print(build_names)
#print(floors)
#print(build_ages)
#print(txn_prices)
#print(saleable_areas)

#driver.quit() #close the chrome browser

In [17]:
# Group the lists to dataframe
Centadata_dataset = pd.DataFrame({
     'Transaction Date': txn_dates,
     'Building Name': build_names,
     'Phase Name': phases,  
     'Floor': floors,
     'Building Age': build_ages,
     'Transaction Price': txn_prices,
     'Saleable Area': saleable_areas 
})

In [18]:
# display the dataframe 
Centadata_dataset

,Transaction Date,Building Name,Phase Name,Floor,Building Age,Transaction Price,Saleable Area
0,06/07/2021,悅海華庭,,27樓,--,740萬,422呎
1,06/07/2021,港島南岸,1期 晉環,15樓,--,1561萬,476呎
2,06/07/2021,碧瑤灣,,15樓,42年,2285萬,"1,363呎"
3,06/07/2021,悅海華庭,,10樓,--,868萬,527呎
4,06/07/2021,漁暉苑,,二十三樓,--,600萬,419呎
...,...,...,...,...,...,...,...
982,03/05/2021,海峰華軒,,39樓,21年,1000萬,567呎
983,03/05/2021,東寶閣,,20樓,32年,574萬,339呎
984,,,,,,"售 $14,600萬","1,720呎"
985,03/05/2021,利東邨,,27樓,34年,195萬,294呎


In [19]:
# export the dataframe to csv file (1st time)
Centadata_dataset.to_csv(OutFile, index=False) 

#export the dataframe to XLS file
#Centadata_dataset.to_excel('centadata_aberdeen.apleichau.xlsx')

In [20]:
# export the dataframe to csv file (append data)
#Centadata_dataset.to_csv('centadata.csv', mode='a', index=False, header=False)